In [1]:
from pyther_classes import *
from pyther_fn import *

In [21]:
# set path
dirPath = '/mnt/c/Users/lvlah/linux/ac_lab/pyther/'

In [22]:
# load interactome
intObj = InteractomefromTSV(dirPath + 'data/stoch-10000_pruned.tsv', 's10k-net')

In [23]:
# load ges
gesObj = anndata.read_csv(dirPath + 'data/stoch-10000_ccle-ges.tsv', delimiter = '\t')

In [128]:
def aREA(ges, intObj):
    gesMat = ges.X
    
    # rank transform the GES
    rankMat = rankdata(gesMat, axis = 0)
    
    # find intersecting genes
    targetSet = intObj.get_targetSet()
    obsNames = gesObj.obs_names.to_list()
    intersectGenes = [value for value in targetSet if value in obsNames]
    
    # reduce regulon matrices
    icMat = intObj.icMat().loc[intersectGenes]
    morMat = intObj.morMat().loc[intersectGenes]
    print(icMat.shape)
    print(morMat.shape)
    
    # prepare the 1-tailed / 2-tailed matrices
    gesInds = [obsNames.index(i) for i in intersectGenes]
    ges2T = rankMat / (rankMat.shape[0] + 1)
    ges1T = abs(ges2T - 0.5) * 2
    ges1T = ges1T + (1 - np.max(ges1T))/2
    ges2TQ = norm.ppf(ges2T[gesInds, :])
    ges1TQ = norm.ppf(ges1T[gesInds, :])
    print(ges2TQ)
    print(ges1TQ)
    
    # 2-tail enrichment
    dES = pd.DataFrame.transpose(pd.DataFrame.mul(icMat, morMat))
    dES = dES.dot(ges2TQ)
    print(dES.shape)
    print(dES)
    
    # 1-tail enrichemnt
    uES = pd.DataFrame.transpose(pd.DataFrame.mul(1 - abs(morMat), icMat))
    uES = uES.dot(ges1TQ)
    print(uES.shape)
    print(uES)
    
    # integrate
    iES = (abs(dES) + uES * (uES > 0)) * np.sign(dES)
    print(iES.shape)
    print(iES)
    # make NES
    nES = iES.mul(intObj.icpVec(), 0)
    
    return(nES)

In [131]:
# run aREA
nesMat = aREA(gesObj, intObj)

(3685, 1927)
(3685, 1927)
[[-0.48596294  1.5502114   1.1032136  ...  1.22007353 -0.49885638
  -0.51005868]
 [-0.02933617 -0.09088967 -0.10775872 ... -0.03591558  1.37630809
   1.3709737 ]
 [-2.09806724 -2.14399687 -2.14745608 ... -2.12047305 -2.11720657
  -2.12211481]
 ...
 [-0.42756494  1.66062004 -0.506711   ... -0.45138976 -0.43573264
  -0.44794347]
 [-1.40219824 -1.46758831  0.87946197 ... -1.44756395 -1.42379902
  -1.43533771]
 [ 1.00690594  1.21752766 -0.08064206 ... -0.00579721  1.04707745
   1.03016178]]
[[-0.32371846  1.16989434  0.61322019 ...  0.76422036 -0.29973809
  -0.27916609]
 [-1.98679503 -1.45750308 -1.36655799 ... -1.89995725  0.95948502
   0.95292468]
 [ 1.80126997  1.85268026  1.85654506 ...  1.82637184  1.82271501
   1.82820946]
 ...
 [-0.43687462  1.30046545 -0.28528885 ... -0.38975091 -0.42055969
  -0.39648118]
 [ 0.99122845  1.07072278  0.30789318 ...  1.04647885  1.01759336
   1.0316338 ]
 [ 0.48479484  0.76098008 -1.51930856 ... -2.59755446  0.53885104
   0.5

In [127]:
nesMat

,0,1,2
AAMP,2.711035,3.977396,2.662886
AATF,-3.244352,3.086070,2.727567
ABCA3,2.692601,-2.148259,2.251413
ABCC3,2.564333,2.510073,2.998940
ABI1,-2.079672,-2.491188,2.350844
...,...,...,...
ZPR1,1.718922,2.590197,2.649971
ZRANB2,3.889095,2.719065,4.659703
ZSCAN31,2.874628,1.237263,2.501337
ZWINT,-2.349889,2.997572,2.790542


In [130]:
gesObj.var_names

Index(['AAACCCAAGACTGAGC-1', 'AAACCCAAGATGCGAC-1', 'AAACCCAAGGCTGAAC-1',
       'AAACCCACAATTGCCA-1', 'AAACCCAGTGGCCCAT-1', 'AAACCCAGTGTGTGGA-1',
       'AAACCCATCCTATGGA-1', 'AAACCCATCCTCTAAT-1', 'AAACCCATCGCAATTG-1',
       'AAACGAAAGCTCGCAC-1',
       ...
       'ACGATGTTCCGATCGG-1', 'ACGATGTTCTATCCAT-1', 'ACGATGTTCTTCGGTC-1',
       'ACGCACGAGATAACAC-1', 'ACGCACGAGCCTCAAT-1', 'ACGCACGCACGCACCA-1',
       'ACGCACGCAGCTGTAT-1', 'ACGCACGCAGGCTACC-1', 'ACGCACGCAGGTCCCA-1',
       'ACGCACGGTTCGGCCA-1'],
      dtype='object', length=500)

In [ ]:
## load data
intObj = InteractomefromTSV('/mnt/c/Users/lvlah/linux/ac_lab/data/pyther_data/test-net.tsv', 'testNet')
gesDF = anndata.read_csv('/mnt/c/Users/lvlah/linux/ac_lab/data/pyther_data/gtex-aorta_ges.tsv', delimiter = '\t')
r = 'ENSG00000000003'

In [9]:
gesObj[:, 0:3].X.shape

(14484, 3)